[Notebook of task](https://github.com/DistributedSystemsGroup/Algorithmic-Machine-Learning/blob/master/Challenges/Anomaly_Detection/anomaly_detection_challenge.ipynb)

### Articles to read about rule based systems

- [RIPPER/JRIP](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.107.2612&rep=rep1&type=pdf)
- [Bayesian Rule Sets](http://jmlr.org/papers/volume18/16-003/16-003.pdf)
- [Scalable Bayesian Rule Lists](https://arxiv.org/pdf/1602.08610.pdf)

In [2]:
# Elementary
import os
import sys
import re
import random
import matplotlib
import implicit
import warnings
from tqdm import tqdm

# For elementary data manipulation
import pandas as pd
import numpy as np

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Import dataframe and cast names
from names import column_names, labels
basepath = "/mnt/datasets/anomaly/"
dataDF = pd.read_csv(basepath + 'data.csv', delimiter=";", header=None, names=column_names)
pure_dataDF = dataDF.drop(labels, axis=1)
anomaliesDF = dataDF.filter(labels, axis=1)

# Data cleaning

## Removing NaNs

In [5]:
anomaliesDF_with_zerNA = anomaliesDF.fillna(0) # Fill NaNs with 0s, considering them as "not an anomaly"
anomaliesDF_with_negNA = anomaliesDF.fillna(-1) # Fill NaNs with -1 considering them as a separate class for the classifier.
pure_dataDF_with_negNA = pure_dataDF.fillna(-1)

X_temp = pure_dataDF_with_negNA.drop('Date', axis=1)

## Casting datatypes

In [6]:
prind 

35

Some of the columns do not contain any data at all (only zeros or NaNs). These will not be usefull for the model, and are therefore removed. Some of the columns contain very little data, and with little variation in the data it is worth considering removing them as well. Some of the columns are in float format, and our rule based model requires them to be integers, so we need to recast them to int with some preprocessing first.

Below is the code used to analyze every column (we only show one example), and the table in the cell below shows the action we take for each column.

In [5]:
sing_ser = X_temp['DeltaSize']

indices_of_non_zero_elements = list(sing_ser.index[np.logical_and(sing_ser != 0.0, sing_ser != -1)])
to_inspect = sing_ser[indices_of_non_zero_elements]

print('Number of non-zero elements:',len(to_inspect), '\n')
start = 100
print(to_inspect[start:start + 5])

Number of non-zero elements: 286471 

100    9.596538e+09
101    1.787984e+09
102    1.496695e+09
103    9.569579e+09
104    2.575049e+09
Name: DeltaSize, dtype: float64


In [6]:
indices_of_non_zero_elements = list(sing_ser.index[sing_ser != 0.0])
to_inspect = sing_ser[indices_of_non_zero_elements]

Below are some short explainations of the different actions, and the table showing which action is taken for the different columns:
* **Remove**: Only zeros and NaNs, do not contribute to model
* **Little_variation**: Little variation in data, may not contribute much to model. The reason for not removing them is that even though they don't vary a lot, they might still be correlated with the different checks.
* **OK**: Can't see anything obviously suspect with the data, and no need for recasting for rule-based model.
* **Direct_recast**: All the numbers are integers, but for some reason they have been casted to float64
* **Format_recast** Values are float or string, and need to be scaled/formatted before they are recast to int

| Column                                |   Action  |
|:--------------------------------------|:----------|
| SessionNumber                         | OK |
| SystemID                              | OK |
| Date                                  | Format_recast |
| HighPriorityAlerts                    | OK |
| Dumps                                 | Direct_recast |
| CleanupOOMDumps                       | Remove |
| CompositeOOMDums                      | Direct_recast |
| IndexServerRestarts                   | Little_variation |
| NameServerRestarts                    | Little_variation |
| XSEngineRestarts                      | Little_variation |
| PreprocessorRestarts                  | Remove |
| DaemonRestarts                        | Remove |
| StatisticsServerRestarts              | Little_variation |
| CPU                                   | Format_recast |
| PhysMEM                               | Format_recast |
| InstanceMEM                           | Format_recast |
| TablesAllocation                      | Format_recast |
| IndexServerAllocationLimit            | Format_recast |
| ColumnUnloads                         | OK |
| DeltaSize                             | Direct_recast |
| MergeErrors                           | Direct_recast |
| BlockingPhaseSec                      | Direct_recast |
| Disk                                  | Format_recast |
| LargestTableSize                      | Direct_recast |
| LargestPartitionSize                  | Direct_recast |
| DiagnosisFiles                        | Direct_recast |
| DiagnosisFilesSize                    | Direct_recast |
| DaysWithSuccessfulDataBackups         | OK |
| DaysWithSuccessfulLogBackups          | OK |
| DaysWithFailedDataBackups             | OK |
| DaysWithFailedfulLogBackups           | OK |
| MinDailyNumberOfSuccessfulDataBackups | Little_variation |
| MinDailyNumberOfSuccessfulLogBackups  | OK |
| MaxDailyNumberOfFailedDataBackups     | OK |
| MaxDailyNumberOfFailedLogBackups      | OK |
| LogSegmentChange                      | Direct_recast |

Below we reformat, and recast the columns listed in the table above. When recasting the columns in format float64 we round them to two decimals and multiply by 100. The column "Date" is in a timestamp format, to cope with this we create two new columns that are day of the week, and minute of the day.

In [11]:
# Removal
X_temp.drop(['CleanupOOMDumps', 'PreprocessorRestarts', 'DaemonRestarts'], axis=1, inplace=True)

# Direct recasting
direct_recast = ['Dumps', 'CompositeOOMDums', 'DeltaSize', 'MergeErrors', 'BlockingPhaseSec', 
                 'LargestTableSize', 'LargestPartitionSize', 'DiagnosisFiles', 'DiagnosisFilesSize', 
                 'LogSegmentChange']
for column in direct_recast:
    X_temp[column] = X_temp[column].astype(np.int64, errors='ignore')

#Format recasting
format_recast = ['CPU', 'PhysMEM', 'InstanceMEM', 'TablesAllocation', 'IndexServerAllocationLimit', 
                    'Disk']
for column in format_recast:
    X_temp[column] = 100*X_temp[column]
    X_temp[column] = X_temp[column].astype(np.int64, errors='ignore')

In [12]:
X_temp.dtypes

SessionNumber                             int64
SystemID                                  int64
Date                                     object
HighPriorityAlerts                        int64
Dumps                                     int64
CompositeOOMDums                          int64
IndexServerRestarts                       int64
NameServerRestarts                        int64
XSEngineRestarts                          int64
StatisticsServerRestarts                  int64
CPU                                       int64
PhysMEM                                   int64
InstanceMEM                               int64
TablesAllocation                          int64
IndexServerAllocationLimit                int64
ColumnUnloads                             int64
DeltaSize                                 int64
MergeErrors                               int64
BlockingPhaseSec                          int64
Disk                                      int64
LargestTableSize                        

Now we have succesfully cast all the datatypes to the format that our rule-based system can handle except for 'Date', which we reformat below. 

In [2]:
column_names = [
    "SessionNumber", 
    "SystemID",
    "Date",
    "HighPriorityAlerts", 
    "Dumps",
    "CleanupOOMDumps",
    "CompositeOOMDums",
    "IndexServerRestarts",
    "NameServerRestarts",
    "XSEngineRestarts",
    "PreprocessorRestarts",
    "DaemonRestarts",
    "StatisticsServerRestarts",
    "CPU",
    "PhysMEM",
    "InstanceMEM",
    "TablesAllocation", 
    "IndexServerAllocationLimit",
    "ColumnUnloads",
    "DeltaSize",
    "MergeErrors",
    "BlockingPhaseSec", 
    "Disk",
    "LargestTableSize",
    "LargestPartitionSize",
    "DiagnosisFiles",
    "DiagnosisFilesSize",
    "DaysWithSuccessfulDataBackups",
    "DaysWithSuccessfulLogBackups",
    "DaysWithFailedDataBackups",
    "DaysWithFailedfulLogBackups",
    "MinDailyNumberOfSuccessfulDataBackups",
    "MinDailyNumberOfSuccessfulLogBackups",
    "MaxDailyNumberOfFailedDataBackups",
    "MaxDailyNumberOfFailedLogBackups",
    "LogSegmentChange",
    "Check1",
    "Check2",
    "Check3",
    "Check4",
    "Check5",
    "Check6",
    "Check7",
    "Check8"
]

labels = ["Check1", "Check2", "Check3", "Check4", "Check5", "Check6", "Check7", "Check8"]

dataDF = pd.read_csv(basepath + 'data.csv', delimiter=";", header=None, names=column_names)
# create pure data DF
puredataDF = dataDF.drop(labels, axis=1)
#create a copy of the label columns for exploration
anomaliesDF = dataDF.filter(labels, axis=1)
# percentage of NA in each label 
print(anomaliesDF.isna().mean().round(4) * 100) 
anomaliesDF_withoutNA = anomaliesDF.fillna(0)

Check1     8.54
Check2     8.53
Check3     7.86
Check4    12.77
Check5    12.21
Check6     2.57
Check7    12.45
Check8     0.02
dtype: float64


In [3]:
anomaliesDF.describe()

,Check1,Check2,Check3,Check4,Check5,Check6,Check7,Check8
count,262520.000000,262545.000000,264463.000000,250384.000000,251997.000000,279647.000000,251309.000000,286979.000000
mean,0.006232,0.033381,0.030602,0.099108,0.012222,0.288120,0.030592,0.010171
std,0.078696,0.179630,0.172236,0.298807,0.109877,0.452888,0.172209,0.100340
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
anomaliesDF_withoutNA.describe()

,Check1,Check2,Check3,Check4,Check5,Check6,Check7,Check8
count,287031.000000,287031.000000,287031.000000,287031.000000,287031.000000,287031.000000,287031.000000,287031.000000
mean,0.005700,0.030533,0.028196,0.086454,0.010731,0.280708,0.026785,0.010170
std,0.075281,0.172050,0.165531,0.281034,0.103031,0.449346,0.161454,0.100331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# percentage of NA in each label 
print(puredataDF.isna().mean().round(4) * 100)

SessionNumber                             0.00
SystemID                                  0.00
Date                                      0.00
HighPriorityAlerts                        0.00
Dumps                                     0.00
CleanupOOMDumps                           0.00
CompositeOOMDums                          0.00
IndexServerRestarts                       0.00
NameServerRestarts                        0.00
XSEngineRestarts                          0.00
PreprocessorRestarts                      0.00
DaemonRestarts                            0.00
StatisticsServerRestarts                  0.00
CPU                                       8.78
PhysMEM                                   7.17
InstanceMEM                               7.71
TablesAllocation                          0.80
IndexServerAllocationLimit                9.21
ColumnUnloads                             0.00
DeltaSize                                 0.07
MergeErrors                               2.69
BlockingPhase